# BoxBunny Deployment

Production deployment notebook for the boxing robot training system.

## Quick Start
1. Run **Environment Setup** (Cell 1)
2. Run **Build Workspace** (Cell 2) - only needed after code changes
3. Run **Launch System** (Cell 3) - starts everything!

## Features
- **Reaction Drill**: Test reflexes with random punch cues (uses YOLO Pose detection)
- **Shadow Sparring**: Practice punch combos
- **Defence Drill**: Block incoming attacks
- **AI Coach**: LLM-powered trash talk and encouragement

## Detection Modes
| Mode | Description | Default |
|------|-------------|---------|
| 🎯 **Pose Detection** | YOLO Pose tracks wrist velocity for reaction timing | ✅ Always On |
| 🎨 **Color Tracking** | Fast glove colors (Red=Left, Green=Right) for display | ✅ Yes |
| 🤖 **Action Model** | ML classifier for punch type prediction | ❌ Experimental |
| 📡 **IMU Sensor** | Glove accelerometer for punch force | ❌ Experimental |

## 1. Environment Setup
Creates the shell environment script. **Run this first every session.**

In [4]:
%%bash
cat > /tmp/boxbunny_env.sh <<'SH'
source ~/miniconda3/etc/profile.d/conda.sh
conda activate boxing_ai
export PYTHONNOUSERSITE=1
export LD_LIBRARY_PATH=$CONDA_PREFIX/lib:$LD_LIBRARY_PATH
export SETUPTOOLS_USE_DISTUTILS=stdlib
source /opt/ros/humble/setup.bash
source /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/install/setup.bash 2>/dev/null || true
SH
echo "✅ Environment ready"

✅ Environment ready


## 2. Build Workspace
Compiles all ROS packages. **Only needed after code changes.**

In [5]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws
colcon build --symlink-install
source install/setup.bash
echo "✅ Build complete"

Starting >>> boxbunny_msgs


--- stderr: boxbunny_msgs
/home/boxbunny/miniconda3/envs/boxing_ai/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
---


Finished <<< boxbunny_msgs [1.10s]
Starting >>> boxbunny_analytics
Starting >>> boxbunny_drills
Starting >>> boxbunny_fusion
Starting >>> boxbunny_gui
Starting >>> boxbunny_imu
Starting >>> boxbunny_llm
Starting >>> boxbunny_vision
Finished <<< boxbunny_analytics [5.18s]
Finished <<< boxbunny_fusion [5.18s]
Finished <<< boxbunny_drills [5.26s]
Finished <<< boxbunny_vision [5.30s]
Finished <<< boxbunny_imu [5.33s]
Finished <<< boxbunny_llm [5.43s]
Finished <<< boxbunny_gui [5.81s]
Starting >>> boxbunny_bringup
Finished <<< boxbunny_bringup [1.98s]

Summary: 9 packages finished [9.62s]
  1 package had stderr output: boxbunny_msgs
✅ Build complete


## 3. 🚀 Launch BoxBunny System (Default Mode)
This is the **main deployment command**. Launches everything with Color Tracking (default, fast & reliable).

**What starts:**
- 📷 RealSense Camera (RGB + Depth)
- 🎨 Color-based Glove Tracker (Red=Jab, Green=Cross)
- 🎯 Reaction Drill Manager
- 📊 Punch Analytics
- 💬 AI Coach (LLM trash talk & encouragement)
- 🖥️ Main GUI (fullscreen)

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws
source install/setup.bash

# Default deployment: Color Tracking + LLM Coach
ros2 launch boxbunny_bringup boxbunny_deploy.launch.py

# Run this command in a separate terminal to see motor commands
# ros2 topic echo /robot/motor_command

[INFO] [launch]: All log files can be found below /home/boxbunny/.ros/log/2026-02-02-14-38-54-147619-ubuntu-42453
[INFO] [launch]: Default logging verbosity is set to INFO
[INFO] [python3-1]: process started with pid [42454]
[INFO] [realsense_glove_tracker-2]: process started with pid [42456]
[INFO] [punch_fusion_node-3]: process started with pid [42458]
[INFO] [reaction_drill_manager-4]: process started with pid [42460]
[INFO] [shadow_sparring_drill-5]: process started with pid [42462]
[INFO] [punch_stats_node-6]: process started with pid [42464]
[INFO] [trash_talk_node-7]: process started with pid [42466]
[INFO] [conda-8]: process started with pid [42468]
[punch_fusion_node-3] [INFO] [1770014334.581043424] [punch_fusion_node]: Punch fusion node ready
[shadow_sparring_drill-5] [INFO] [1770014334.592296613] [shadow_sparring_drill]: Loaded 6 drills from /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/install/boxbunny_drills/share/boxbunny_drills/config/drill_definitions.yaml

TypeError: %d format: a real number is required, not NoneType

[ERROR] [trash_talk_node-7]: process has died [pid 42466, exit code 1, cmd '/home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/install/boxbunny_llm/lib/boxbunny_llm/trash_talk_node --ros-args --params-file /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/install/boxbunny_llm/share/boxbunny_llm/config/llm.yaml'].
[python3-1] Traceback (most recent call last):
[python3-1]   File "/home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/action_prediction/tools/inference/live_infer_rgbd.py", line 1847, in <module>
[python3-1]     main()
[python3-1]   File "/home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/action_prediction/tools/inference/live_infer_rgbd.py", line 1804, in main
[python3-1]     HeadlessInference(
[python3-1]   File "/home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws/action_prediction/tools/inference/live_infer_rgbd.py", line 1400, in __init__
[python3-1]     self._loop()
[python3-1]   File "/home/boxbunny/Desktop/doomsday_integratio

---
## 🧪 Experimental Modes (Optional)

### 4a. Launch with AI Action Model
Uses YOLO Pose + ML model for more accurate punch detection. **Requires GPU.**

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws
source install/setup.bash

# Experimental: Action Model mode (GPU required)
ros2 launch boxbunny_bringup boxbunny_deploy.launch.py detection_mode:=action

### 4b. Launch with IMU Sensor
Adds glove-mounted accelerometer for punch force measurement. **Requires MPU6050 hardware.**

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws
source install/setup.bash

# Experimental: With IMU sensor
ros2 launch boxbunny_bringup boxbunny_deploy.launch.py enable_imu:=true

### 4c. Full Experimental Mode (Action Model + IMU)

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws
source install/setup.bash

# Full experimental: Action Model + IMU
ros2 launch boxbunny_bringup boxbunny_deploy.launch.py detection_mode:=action enable_imu:=true

---
## 📖 System Architecture Reference

### ROS2 Nodes Overview
| Node | Package | Purpose | Topic/Service |
|------|---------|---------|---------------|
| `realsense2_camera_node` | realsense2_camera | RGB-D camera driver | `/camera/color/image_raw`, `/camera/depth/image_rect_raw` |
| `realsense_glove_tracker` | boxbunny_vision | Color-based punch detection | Publishes: `glove_detections`, `punch_events` |
| `punch_fusion_node` | boxbunny_fusion | Combines vision + IMU data | Publishes: `punch_events` (fused) |
| `reaction_drill_manager` | boxbunny_drills | Manages drill state machine | `drill_state`, `drill_summary`, Services: `start_stop_drill` |
| `punch_stats_node` | boxbunny_analytics | Tracks punch statistics | Subscribes to punch events |
| `trash_talk_node` | boxbunny_llm | AI Coach (encouragement/trash talk) | `trash_talk`, Service: `llm/generate` |
| `mpu6050_node` | boxbunny_imu | IMU sensor driver (experimental) | `imu/data`, `imu/debug` |
| `imu_punch_classifier` | boxbunny_imu | Classifies punches from IMU (experimental) | `imu/punch` |

### Key Topics for Integration
```
# Get real-time punch events
ros2 topic echo /punch_events

# Get player height (after calibration)
ros2 topic echo /player_height

# Get AI coach messages
ros2 topic echo /trash_talk

# Get drill state
ros2 topic echo /drill_state
```

### LLM Coach Integration
The AI Coach (`trash_talk_node`) provides:
- **Encouragement**: Positive feedback after good punches
- **Trash Talk**: Playful taunts to motivate the user
- **Analysis**: Performance insights after drills
- **Coaching**: Technical tips based on punch patterns

To request custom LLM responses:
```python
# From your code:
from boxbunny_msgs.srv import GenerateLLM
# modes: "coach", "encourage", "trash", "analysis"
```

---
## 🔧 Troubleshooting & Utilities

### Check Node Status

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
echo "=== Active ROS2 Nodes ==="
ros2 node list 2>/dev/null || echo "No ROS2 nodes running"
echo ""
echo "=== Active Topics ==="
ros2 topic list 2>/dev/null || echo "No topics available"

### Kill All BoxBunny Processes
Run this if the system gets stuck or you need a clean restart.

In [ ]:
%%bash
echo "Killing BoxBunny processes..."
pkill -f "boxbunny" || true
pkill -f "realsense2_camera" || true
pkill -f "live_infer_rgbd" || true
pkill -f "gui_main" || true
sleep 1
echo "✅ All processes killed"

### Test RealSense Camera
Quick check to verify the camera is connected and working.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh
echo "=== RealSense Camera Check ==="
rs-enumerate-devices 2>/dev/null | head -20 || echo "❌ RealSense SDK not found or no camera connected"

### Launch GUI Only (Standalone)
If you need to restart just the GUI without restarting the ROS nodes. **Note:** Uses conda Python directly due to PySide6/Qt compatibility.

In [ ]:
%%bash
source /tmp/boxbunny_env.sh

# GUI must run with conda Python (not ros2 run) due to Qt ABI requirements
cd /home/boxbunny/Desktop/doomsday_integration/boxing_robot_ws
python src/boxbunny_gui/boxbunny_gui/gui_main.py &
echo "✅ GUI launched"

---
## 📋 Development Notes

### Known Issues & Solutions
| Issue | Solution |
|-------|----------|
| GUI crashes with Qt error | Run GUI directly with `python gui_main.py` instead of `ros2 run` |
| Camera not detected | Run `cleanup_camera.sh` then reconnect USB |
| LLM responses slow | Ensure Ollama is running: `ollama serve` |
| Action model not loading | Check GPU memory, may need to reduce batch size |

### File Locations
- **Main GUI**: `src/boxbunny_gui/boxbunny_gui/gui_main.py`
- **Launch Files**: `src/boxbunny_bringup/launch/`
- **Vision Nodes**: `src/boxbunny_vision/boxbunny_vision/`
- **Drill Logic**: `src/boxbunny_drills/boxbunny_drills/`
- **Models**: `models/checkpoints/` (YOLO), `models/action_prediction_model/` (Action classifier)